<a href="https://colab.research.google.com/github/dariakrtn/AI_2022/blob/main/lab_3_NUMBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. *Импорт библиотек и вспомогательные функции/классы*

In [16]:
import numpy as np
from mpl_toolkits import mplot3d
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from typing import Tuple, Mapping

In [17]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import math

In [18]:
import numba
from numba import jit

In [19]:
import timeit

Реализация шедулера

In [20]:
class Lr:
  def __init__(self, lr) -> None:
    self.lr = lr

  def step(self):
    self.lr -= 0.0001

# 1. Функция Матьяса 

$0.26 (x^2 + y^2) - 0.48  x y$

In [21]:
def f_mat (x: np.array, y:np.array) -> np.float128:
  return 0.26*(x*x +y*y)-0.48*x*y

In [22]:
def dx_mat (x: np.array, y:np.array) -> np.float128:
  return 0.52 *x -0.48*y

In [23]:
def dy_mat(x: np.array, y:np.array) -> np.float128:
  return 0.52 * y - 0.48*x

# 2. *Градиентный метод* 


In [31]:
def GD( f: Mapping, dx: Mapping, dy: Mapping, point: np.array, lr: float, iter: int, l: float, r:float):
  '''
  f - входная функция
  dx, - производная функции по х
  dy, - производная функция по y
  point - точка входа
  lr, -  learning rate шаг
  iter, - кол-во итераций
  l, r - левая и правая граница x, y
  '''
  lr = Lr(lr)

  for _ in range(iter):
    gx= dx(point[0], point[1])
    gy = dy(point[0], point[1])

    point[0] = point[0] - gx * lr.lr
    point[1] = point[1] - gy * lr.lr

  return point

In [32]:
point = np.array([-10,10])
and_point = GD(f_mat, dx_mat, dy_mat, point, 0.001, 10, -10, 10 )
print(f_mat(point[0], point[1]), and_point)
%timeit -n100 GD(f_mat, dx_mat, dy_mat, point, 0.001, 10, -10, 10 )

0.0 [0 0]
70.5 µs ± 2.67 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# 3. Функция Матьяса c Numba 


In [26]:
@numba.njit(fastmath=True)
def f_mat (x: np.array, y:np.array) -> np.float128:
  return 0.26*(x*x +y*y)-0.48*x*y

In [27]:
@numba.njit(fastmath=True)
def dx_mat (x: np.array, y:np.array) -> np.float128:
  return 0.52 *x -0.48*y

In [28]:
@numba.njit(fastmath=True)
def dy_mat(x: np.array, y:np.array) -> np.float128:
  return 0.52 * y - 0.48*x

# 4. *Градиентный метод c Numba* 


In [33]:
@numba.njit(fastmath=True)
def NUMBA_GD( point: np.array, lr: float, iter: int, l: float, r:float):
  '''
  point - точка входа
  lr, -  learning rate шаг
  iter, - кол-во итераций
  l, r - левая и правая граница x, y
  '''

  for _ in range(iter):
    gx= dx_mat(point[0], point[1])
    gy = dy_mat(point[0], point[1])

    point[0] = point[0] - gx * lr
    point[1] = point[1] - gy * lr
    lr -= 0.0001

  return point

In [34]:
point = np.array([-10,10])
and_point = NUMBA_GD(point, 0.001, 10, -10, 10 )
print(f_mat(point[0], point[1]), and_point)
%timeit -n100 NUMBA_GD(point, 0.001, 10, -10, 10 )

0.0 [0 0]
941 ns ± 75.6 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)
